In [1]:
"""
Create log file that can be populated automatically each time data is saved
@author: Ines
"""
import os
import json
from datetime import datetime
import pickle
from collections import defaultdict
import sys

In [2]:
# Data to be written

log = defaultdict(list)

# current date
now = datetime.now() # current date and time
date_time = now.strftime("%m-%d-%Y")

# List all .ipynb files in the current working directory
notebook_files = [f for f in os.listdir() if f.endswith('.ipynb')]
notebook_name = notebook_files[0]

log_entry = {"data_filename": "metadata_log.json",
    "script_name": str(notebook_name),
    "timestamp": date_time
}
# First entry
log[0] = log_entry
# Where to put json
log_file_path =  '/home/ines/repositories/representation_learning_variability/DATA/'
 
# Serializing json
# log_file = json.dumps(log, indent=4)
log_file = json.dumps(log)

# Writing to sample.json
with open(log_file_path+"metadata_log.json", "w") as outfile:
    outfile.write(log_file)

 
# Opening JSON file
with open(log_file_path+'metadata_log.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)

In [2]:
log_file_path =  '/home/ines/repositories/representation_learning_variability/DATA/'
 
# Opening JSON file
with open(log_file_path+'metadata_log.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)

### Automatically update log when saving
This function is copied in data_processing.py and should be called from there

In [8]:
def save_and_log(file_to_save, filename, file_format, save_path, script_name):

    # current date
    now = datetime.now() # current date and time
    date_time = now.strftime("%m-%d-%Y")
    
    # First, save file in desired location and with desired format
    os.chdir(save_path)
    if file_format == 'pickle':
        pickle.dump(file_to_save, open(filename+date_time, "wb"))
    elif file_format == 'parquet':
        assert isinstance(file_to_save, pd.DataFrame) 
        file_to_save.to_parquet(filename+date_time, compression='gzip')  
    else: 
        sys.exit("File format not implemented")        

    # Open log file json
    log_file_path =  '/home/ines/repositories/representation_learning_variability/DATA/' 
    with open(log_file_path + 'metadata_log.json', 'r') as openfile:
        # Reading from json file
        metadata_log = json.load(openfile)
    
    # # Then create entry for the log file
    # files = [f for f in os.listdir() if f.endswith('.ipynb')]
    # script_name = files[0]
    
    # Populate new entry
    new_log_entry = {
    "data_filename": filename,
    "script_name": script_name,
    "timestamp": date_time
    }
    
    # Update log dict
    order_last_entry = int(list(metadata_log.keys())[-1])
    metadata_log[order_last_entry+1] = new_log_entry
    updated_json = json.dumps(metadata_log)

    # Overwrite json log file
    with open(log_file_path+"metadata_log.json", "w") as outfile:
        outfile.write(updated_json)

    return metadata_log
